In [1]:
from kaiseki import lattice_kai
from kaiseki import kakunou_kai
from kaiseki import Short_Algo_kai
from kaiseki import Long_kaiseki
from dir_sci import Short_sci
from nume_ex_prm import Short_FISTA
from nume_ex_prm import Short_qNewton
import time
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import scipy.sparse as spsp
import matplotlib.pyplot as plt
import csv
import os

from scipy.spatial import distance
from scipy.optimize import linprog
from scipy.optimize import minimize
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

## 長期の均衡解を求める

In [3]:
E_list = [1, 2, 5, 8, 10]
Col_list = [5, 8, 10]
for Col in Col_list:
    
    for E in E_list:
        # Scaling * tが格子の最小距離となる。
        Scaling = 0.15

        city_network = lattice_kai.make_lattice(Col)
        
        K = Col * Col
    
        M = 0.1 * K
        N = 0.1 * K
    
        m_per = M / K
        m0 = np.array([m_per] * K)
        n0 = np.full((K, K), N / (K * K))

        # 通勤費用パラメータ
        t = 0.1

        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        #パラメータ設定
        prm = Long_kaiseki.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)

        long = Long_kaiseki.Long(prm)
        
        m_true, n_true, RW = long.solve(m0, n0, long_itr=1000, err_long=1e-5)
        
        # print("len(m_true):", len(m_true))
        # print("len(n_true):", len(n_true))
        # print("len(n_true.flatten()):", len(n_true.flatten()))

        true_path_f, true_path_h = kakunou_kai.kakunou_true_long(m_true, n_true, prm)
        
        print("true_path_f:", true_path_f)
        print("true_path_h:", true_path_h)

long_max_value: 76
Lyapunov: -1.929016946178308e-10
true_path_f: C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/shortlong/kaisekikai/theta=5.0/dist=0.15/E=1/v_proj=0.1/K=5^2/truevalue/jikken_F.xlsx
true_path_h: C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/shortlong/kaisekikai/theta=5.0/dist=0.15/E=1/v_proj=0.1/K=5^2/truevalue/jikken_H.xlsx
long_max_value: 76
Lyapunov: -1.929016946178308e-10
true_path_f: C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/shortlong/kaisekikai/theta=5.0/dist=0.15/E=2/v_proj=0.1/K=5^2/truevalue/jikken_F.xlsx
true_path_h: C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/shortlong/kaisekikai/theta=5.0/dist=0.15/E=2/v_proj=0.1/K=5^2/truevalue/jikken_H.xlsx
long_max_value: 76
Lyapunov: -1.929025827962505e-10
true_path_f: C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/shortlong/kaisekikai/theta=5.0/dist=0.15/E=5/v_proj=0.1/K=5^2/truevalu

## FISTAとの比較

In [6]:
E_list = [1, 2, 5, 8, 10]
Col_list = [5, 8, 10]
for Col in Col_list:
    
    for E in E_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        K = Col * Col
    
        M = 0.1 * K
        N = 0.1 * K

        dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
        fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_F.xlsx')
        dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
        fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_H.xlsx')

        m_fixed = np.array(dF_long['m'].tolist())
        n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

        # m_per = M / K
        # m_fixed = np.array([m_per] * K)
        # n_fixed = np.full((K, K), N / (K * K))
        
        Scaling = 3 / Col

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.5
        p_proj = 0.1

        #パラメータ設定
        prm = Short_qNewton.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
        algprm = Short_qNewton.Algo_Parameter(L, eta, p_proj)

        dF_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_F.xlsx')
        dF_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_F.xlsx')
        dH_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_H.xlsx')
        dH_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_H.xlsx')

        R_FISTA = np.array(dF_FISTA['R'].tolist())
        R_kaiseki = np.array(dF_kaiseki['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_FISTA - R_kaiseki) / R_kaiseki)))
        print("R_FISTA:", np.mean(R_FISTA))
        print("R_kaiseki:", np.mean(R_kaiseki))

        W_FISTA = np.array(dF_FISTA['W'].tolist())
        W_kaiseki = np.array(dF_kaiseki['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_FISTA - W_kaiseki) / W_kaiseki)))
        print("W_FISTA:", np.mean(W_FISTA))
        print("W_kaiseki:", np.mean(W_kaiseki))

        pi_FISTA = np.array(dF_FISTA['pi'].tolist())
        pi_kaiseki = np.array(dF_kaiseki['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_FISTA - pi_kaiseki) / pi_kaiseki)))
        print("pi_FISTA:", np.mean(pi_FISTA))
        print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_FISTA = np.array(dH_FISTA['v'].tolist()).flatten()
        v_kaiseki = np.array(dH_kaiseki['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_FISTA)):
            if v_FISTA[i] == 0:  # ゼロ割りを防ぐための処理
                if v_kaiseki[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_FISTA[i] - v_kaiseki[i]) / v_kaiseki[i]))
            
        # print("v_err:", v_err)
        
        # print("v_rel_err:", np.mean(v_err))
        print("v_FISTA:", v_FISTA)
        print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = Short_Algo.Short(prm, algprm, m_fixed, n_fixed)
        RW_FISTA = np.concatenate((R_FISTA, W_FISTA))
        dRdW_FISTA = short.short_dual_df(RW_FISTA)
        
        dR_FISTA = dRdW_FISTA[:K]
        dW_FISTA = dRdW_FISTA[K:]
        
        print("dR_FISTA:", np.mean(dR_FISTA))
        print("dW_FISTA:", np.mean(dW_FISTA))
        
        short = Short_Algo.Short(prm, algprm, m_fixed, n_fixed)
        RW_kaiseki = np.concatenate((R_kaiseki, W_kaiseki))
        dRdW_kaiseki = short.short_dual_df(RW_kaiseki)
        
        dR_kaiseki = dRdW_kaiseki[:K]
        dW_kaiseki = dRdW_kaiseki[K:]
        
        print("dR_kaiseki:", np.mean(dR_kaiseki))
        print("dW_kaiseki:", np.mean(dW_kaiseki))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

E=10, K=10^2
R_rel_err: 5.261603341924876e-10
R_FISTA: 0.0800000000344006
R_kaiseki: 0.08000000000000003
W_rel_err: 4.378942910757509e-09
W_FISTA: 0.08006262825524074
W_kaiseki: 0.08006262800727271
pi_rel_err: 1.8737220564774872e-10
pi_FISTA: 10.334925457728074
pi_kaiseki: 10.334925459157002
v_FISTA: [1.33531097 1.30160733 1.2692404  ... 1.2692404  1.30160733 1.33531097]
v_kaiseki: [1.33531096 1.30160733 1.2692404  ... 1.2692404  1.30160733 1.33531096]
dR_FISTA: 4.627841715398873e-10
dW_FISTA: 3.2857668047503454e-09
dR_kaiseki: 2.2204460492503132e-17
dW_kaiseki: 5.440092820663267e-17
E=10, K=20^2
R_rel_err: 9.431667487491339e-12
R_FISTA: 0.07999999999915106
R_kaiseki: 0.07999999999999999
W_rel_err: 1.2250244779714572e-10
W_FISTA: 0.07941719503721612
W_kaiseki: 0.07941719503096581
pi_rel_err: 1.3252093232150811e-12
pi_FISTA: 39.89840176392494
pi_kaiseki: 39.89840176395589
v_FISTA: [1.33180079 1.32157056 1.31116495 ... 1.31116495 1.32157056 1.33180079]
v_kaiseki: [1.33180079 1.32157056 1

## qNewtonとの比較

In [6]:
E_list = [5]
Col_list = [10]
for Col in Col_list:
    
    K = Col * Col
    
    M = 0.1 * K
    N = 0.1 * K
    
    dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_F.xlsx')
    dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_H.xlsx')

    m_fixed = np.array(dF_long['m'].tolist())
    n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

    # m_per = M / K
    # m_fixed = np.array([m_per] * K)
    # n_fixed = np.full((K, K), N / (K * K))
    
    for E in E_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        Scaling = 3 / Col

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.2
        p_proj = 0.1

        #パラメータ設定
        prm = Short_qNewton.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
        algprm = Short_qNewton.Algo_Parameter(L, eta, p_proj)

        dF_qNewton = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/qNewton/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_F.xlsx')
        dF_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_F.xlsx')
        dH_qNewton = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/qNewton/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_H.xlsx')
        dH_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_H.xlsx')

        R_qNewton = np.array(dF_qNewton['R'].tolist())
        R_kaiseki = np.array(dF_kaiseki['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_qNewton - R_kaiseki) / R_kaiseki)))
        # print("R_qNewton:", np.mean(R_qNewton))
        # print("R_kaiseki:", np.mean(R_kaiseki))

        W_qNewton = np.array(dF_qNewton['W'].tolist())
        W_kaiseki = np.array(dF_kaiseki['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_qNewton - W_kaiseki) / W_kaiseki)))
        # print("W_qNewton:", np.mean(W_qNewton))
        # print("W_kaiseki:", np.mean(W_kaiseki))

        pi_qNewton = np.array(dF_qNewton['pi'].tolist())
        pi_kaiseki = np.array(dF_kaiseki['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_qNewton - pi_kaiseki) / pi_kaiseki)))
        # print("pi_qNewton:", np.mean(pi_qNewton))
        # print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_qNewton = np.array(dH_qNewton['v'].tolist()).flatten()
        v_kaiseki = np.array(dH_kaiseki['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_qNewton)):
            if v_qNewton[i] == 0:  # ゼロ割りを防ぐための処理
                if v_kaiseki[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_qNewton[i] - v_kaiseki[i]) / v_kaiseki[i]))
            
        # print("v_err:", v_err)
        
        print("v_rel_err:", np.mean(v_err))
        # print("v_qNewton:", v_qNewton)
        # print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = Short_qNewton.Short(prm, algprm, m_fixed, n_fixed)
        RW_qNewton = np.concatenate((R_qNewton, W_qNewton))
        dRdW_qNewton = short.short_dual_df(RW_qNewton)
        
        dR_qNewton = dRdW_qNewton[:K]
        dW_qNewton = dRdW_qNewton[K:]
        
        print("dR_qNewton:", np.mean(dR_qNewton))
        print("dW_qNewton:", np.mean(dW_qNewton))

        RW_kaiseki = np.concatenate((R_kaiseki, W_kaiseki))
        dRdW_kaiseki = short.short_dual_df(RW_kaiseki)
        
        dR_kaiseki = dRdW_kaiseki[:K]
        dW_kaiseki = dRdW_kaiseki[K:]
        
        print("dR_kaiseki:", np.mean(dR_kaiseki))
        print("dW_kaiseki:", np.mean(dW_kaiseki))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

E=5, K=10^2
R_rel_err: 5.132872891147867e-09
W_rel_err: 3.2862844886454863e-09
pi_rel_err: 2.871722204894149e-10
v_rel_err: 3.201998648122162e-09
dR_qNewton: 2.8883800906598367e-10
dW_qNewton: -1.3329561066033335e-11
dR_kaiseki: 2.2204460492503132e-17
dW_kaiseki: 1.5543122344752193e-17
